In [ ]:
import json
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd


In [ ]:

def save_json(data,name):
    with open(name+'.json', 'w',encoding='utf8') as f:
        json.dump(data, f,ensure_ascii=False)
        
def save_jsonl(data,name):
    with open(name+'.jsonl', 'w') as f:
        for entry in data:
            json.dump(entry, f)
            outfile.write('\n')

def load_json(path):
    f = open (path, "r")
    data = json.loads(f.read())
    f.close()
    return data

def load_jsonl(path):
    f = open(path,"r").readlines()
    data=[]
    for i in f:
        d = json.loads(i)
        data.append(d)
    return data

# Dataset 1

In dataset-1 we have 20 .txt files. Each file has a word and corresponding label in each line. We will extract all of them and make lists. If the number of space separated parts and number of labels are not same we will skip those sentences. If a token contains a name or part of a name it is labeled 1 otherwise 0.

In [ ]:

all_sen = []
all_leb = []
all_leb_per = []

for k in range(20):

    f = open("/path/to/Dataset_1/"+str(k+1)+".txt", "r")
    data = f.readlines()
    data[0]=data[0][1:]

    i = 0

    while(i<len(data)):

        if data[i]=="\n":
            i = i+1
            continue
        else:
            sen = ""
            leb = []
            leb_per = []
            while(i<len(data) and data[i]!="\n"):
                sen_leb = data[i].split("\t")
                sen = sen +" "+sen_leb[0].replace(" ","")
                if sen_leb[1][:-1][-3:]=="PER":
                    leb_per.append(1)
                else:
                    leb_per.append(0)
                leb.append(sen_leb[1][:-1])
                i = i+1

            all_leb_per.append(leb_per)
            all_sen.append(sen[1:])
            all_leb.append(leb)


# Dataset 2

In dataset-2 there is some errors in labeling. There are some sentences which contains multiple names but their labels are not given at same label. The sentence is repeated multiple times and each time the only one name label is given. First we will have to merge all of them. I

In [ ]:
data_2 = load_jsonl("/path/to/Dataset_2.jsonl")

# Making a csv and groupig multiple labels
all_sen_bad = [data_2[i][0] for i in range(len(data_2))]
all_leb_bad = [data_2[i][1] for i in range(len(data_2))]

df = pd.DataFrame({"sen":all_sen_bad,"leb":all_leb_bad})
df_2 = df.groupby("sen").agg(list).reset_index()

# Merging the labels
bad = []
new_leb = []
new_sen = []
total_leb = ['L-PERSON', 'I-PERSON', 'L-ORG', 'B-PERSON', 'U-DATE', 'I-GPE', 'U-ORG', 'L-GPE', 'U-GPE', 'U-PERSON', 'B-GPE', 'I-ORG', 'B-LAW', 'L-LAW', 'I-LAW', 'B-ORG']
for i in range(len(df_2)):
    li = df_2.iloc[i]["leb"]
    new_sen.append(df_2.iloc[i]["sen"])
    if len(li)==1:
        new_leb.append(li[0])
    else:
        bad.append(i)
        corr_leb = li[0]
        dummy = [[k[j] for k in li] for j in range(len(li[0]))]
        for t in range(len(corr_leb)):
            for tt in range(len(li)):
                if dummy[t][tt] in total_leb:
                    corr_leb[t]=dummy[t][tt]
                    
                    break
        
        new_leb.append(corr_leb)
        

# declaring new data
data_2 = [[new_sen[i],new_leb[i]] for i in range(len(new_sen))]


Now we will repeat similar process as dataset-1

In [ ]:
all_sen_2 = []
all_leb_2 = []
all_leb_per_2 = []

for i in range(len(data_2)):
    sen = data_2[i][0].replace(","," ,").replace("।"," ।").replace(":"," :").replace("-"," - ").replace("(","( ").replace(")"," )").replace("'"," '").replace(";"," ;").replace("  "," ")
    if len(sen.split(" "))!=len(data_2[i][1]):
        continue
    all_sen_2.append(sen)
    all_leb_2.append(data_2[i][1])
    all_leb_per_2.append([1 if i[-6:]=='PERSON' else 0 for i in data_2[i][1]])


# Full Data

Now we will add two datasets and shuffle all of them.

In [ ]:
all_sen_20 = all_sen+all_sen_2
all_leb_20 = all_leb+all_leb_2
all_leb_per_20 = all_leb_per+all_leb_per_2

import random
idxs = np.arange(len(all_sen_20))
random.shuffle(idxs)

all_sen_20 = [all_sen_20[i] for i in idxs]
all_leb_20 = [all_leb_20[i] for i in idxs]
all_leb_per_20 = [all_leb_per_20[i] for i in idxs]



We will now check if the labels and sentences are correlated with each other.

In [ ]:
for i in range(len(all_sen_20)):
    if (len(all_sen_20[i].split(" "))==len(all_leb_20[i])==len(all_leb_per_20[i]))!=True:
        print("ok")

Here, we will separate the examples which contain at least one name and which contain no name. It is important to make the split balanced.

In [ ]:
all_sen_201 = [] #2612
all_leb_201 = []
all_leb_per_201 = []

all_sen_202 = [] #6682
all_leb_202 = []
all_leb_per_202 = []

for i in range(len(all_sen_20)):
    if 1 in all_leb_per_20[i]:
        all_sen_201.append(all_sen_20[i])
        all_leb_201.append(all_leb_20[i])
        all_leb_per_201.append(all_leb_per_20[i])
    else:
        all_sen_202.append(all_sen_20[i])
        all_leb_202.append(all_leb_20[i])
        all_leb_per_202.append(all_leb_per_20[i])
        

Finally we will split the data in three parts. They are - Train, Validation and Test. After that we will save the data in json files.

In [ ]:
train_sen = all_sen_201[:2089]*3+all_sen_202[:5345] # a multiplication factor of 3 is used to make it balanced
train_leb = all_leb_201[:2089]*3+all_leb_202[:5345]
train_leb_per = all_leb_per_201[:2089]*3+all_leb_per_202[:5345]

valid_sen = all_sen_201[2089:2350]+all_sen_202[5345:6013]
valid_leb = all_leb_201[2089:2350]+all_leb_202[5345:6013]
valid_leb_per = all_leb_per_201[2089:2350]+all_leb_per_202[5345:6013]

test_sen = all_sen_201[2350:]+all_sen_202[6013:]
test_leb = all_leb_201[2350:]+all_leb_202[6013:]
test_leb_per = all_leb_per_201[2350:]+all_leb_per_202[6013:]

train_dict = {"sentences":train_sen, "entity":train_leb, "label":train_leb_per}
valid_dict = {"sentences":valid_sen, "entity":valid_leb, "label":valid_leb_per}
test_dict = {"sentences":test_sen, "entity":test_leb, "label":test_leb_per}


In [ ]:
save_json(train_dict,"train")
save_json(valid_dict,"valid")
save_json(test_dict,"test")